<div style='text-align: center;'>
    <img src='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQzJzIHdangJTrH2mFXFgsLjuLCjpfXXwbxg&usqp=CAU' width='100'/>
    <h1>Sharif University of Technology</h1>
    <h2>Natural Language Processing</h2>
    <h3>Final Project</h3>
    <h4>Spoiler classification and summary generation</h4>
    <p><strong>Authors:</strong> Abolfazl eshagh</p>
</div>
<hr/>


In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Trainer, TrainingArguments
import gc
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

2024-08-09 22:49:47.150814: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-09 22:49:47.150961: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 22:49:47.272051: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# movie_details = pd.read_json('IMDB_movie_details.json', lines=True)
reviews = pd.read_json('/kaggle/input/imdb-spoiler-dataset/IMDB_reviews.json', lines=True)

In [4]:
reviews = reviews[reviews['review_summary'] != '']
reviews

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
573908,8 August 1999,tt0139239,ur0100166,False,"Go is wise, fast and pure entertainment. Assem...",10,The best teen movie of the nineties
573909,31 July 1999,tt0139239,ur0021767,False,"Well, what shall I say. this one´s fun at any ...",9,Go - see the movie
573910,20 July 1999,tt0139239,ur0392750,False,"Go is the best movie I have ever seen, and I'v...",10,It's the best movie I've ever seen
573911,11 June 1999,tt0139239,ur0349105,False,Call this 1999 teenage version of Pulp Fiction...,3,Haven't we seen this before?


In [5]:
review1 = reviews.loc[1]
# plot1 = movie_details.loc[1]
print(review1)
print()
# print(plot1)

review_date                                        6 September 2000
movie_id                                                  tt0111161
user_id                                                   ur0842118
is_spoiler                                                     True
review_text       The Shawshank Redemption is without a doubt on...
rating                                                           10
review_summary           Simply amazing. The best film of the 90's.
Name: 1, dtype: object



In [6]:
# movie_details = movie_details.drop(columns = ['movie_id', 'duration', 'genre', 'rating', 'release_date'])
reviews = reviews.drop(columns = ['review_date', 'movie_id', 'user_id', 'is_spoiler', 'rating'])
reviews

,review_text,review_summary
0,"In its Oscar year, Shawshank Redemption (writt...",A classic piece of unforgettable film-making.
1,The Shawshank Redemption is without a doubt on...,Simply amazing. The best film of the 90's.
2,I believe that this film is the best story eve...,The best story ever told on film
3,"**Yes, there are SPOILERS here**This film has ...",Busy dying or busy living?
4,At the heart of this extraordinary movie is a ...,"Great story, wondrously told and acted"
...,...,...
573908,"Go is wise, fast and pure entertainment. Assem...",The best teen movie of the nineties
573909,"Well, what shall I say. this one´s fun at any ...",Go - see the movie
573910,"Go is the best movie I have ever seen, and I'v...",It's the best movie I've ever seen
573911,Call this 1999 teenage version of Pulp Fiction...,Haven't we seen this before?


In [41]:
from sklearn.model_selection import train_test_split
plot_dataset = reviews[['review_text', 'review_summary']]

X = reviews['review_text']
y = reviews['review_summary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# lower the size of the dataset
X_train, _, y_train, _ = train_test_split(X_train, y_train, test_size=0.75, random_state=42)
X_test, _, y_test, _ = train_test_split(X_test, y_test, test_size=0.75, random_state=42)
X_val, _, y_val, _ = train_test_split(X_val, y_val, test_size=0.75, random_state=42)

# lower the size of the dataset
X_train, _, y_train, _ = train_test_split(X_train, y_train, test_size=0.9, random_state=42)
X_test, _, y_test, _ = train_test_split(X_test, y_test, test_size=0.9, random_state=42)
X_val, _, y_val, _ = train_test_split(X_val, y_val, test_size=0.9, random_state=42)

train_set = pd.DataFrame({'review_text': X_train, 'review_summary': y_train})
test_set = pd.DataFrame({'review_text': X_test, 'review_summary': y_test})
val_set = pd.DataFrame({'review_text': X_val, 'review_summary': y_val})


In [10]:
len(train_set), len(test_set), len(val_set)

(401737, 86087, 86087)

In [42]:
len(train_set), len(test_set), len(val_set)

(10043, 2152, 2152)

In [20]:
# Load BART tokenizer and model
model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

In [43]:
# Tokenize data
train_encodings = tokenizer(train_set['review_text'].tolist(), truncation=True, padding="max_length", max_length=128, return_tensors="pt")
train_labels = tokenizer(train_set['review_summary'].tolist(), truncation=True, padding="max_length", max_length=32, return_tensors="pt")
val_encodings = tokenizer(val_set['review_text'].tolist(), truncation=True, padding="max_length", max_length=128, return_tensors="pt")
val_labels = tokenizer(val_set['review_summary'].tolist(), truncation=True, padding="max_length", max_length=32, return_tensors="pt")
test_encodings = tokenizer(test_set['review_text'].tolist(), truncation=True, padding="max_length", max_length=128, return_tensors="pt")
test_labels = tokenizer(test_set['review_summary'].tolist(), truncation=True, padding="max_length", max_length=32, return_tensors="pt")

In [44]:
# Create dataset class
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels['input_ids'][idx].clone().detach()
        return item

In [45]:
train_dataset_prepared = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)
test_dataset = ReviewDataset(test_encodings, test_labels)

In [46]:
# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1000,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [47]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_prepared,
    eval_dataset=val_dataset,
)

In [48]:
trainer.train()

Step,Training Loss,Validation Loss
1000,1.765400,1.196940
2000,1.238500,1.125609
3000,1.159900,1.092601
4000,1.098300,1.061806
5000,1.063100,1.026974


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=5022, training_loss=1.264352548554427, metrics={'train_runtime': 1122.4689, 'train_samples_per_second': 8.947, 'train_steps_per_second': 4.474, 'total_flos': 2720528939876352.0, 'train_loss': 1.264352548554427, 'epoch': 1.0})

In [49]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [50]:
# Save model and tokenizer
# model.save_pretrained('./saved_model')
# tokenizer.save_pretrained('./saved_model')
torch.save(model.state_dict, 'bart_review_summary_generation1.pth')

In [54]:
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# val_dataset = PlotDataset(test_encodings, test_labels)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

all_predictions = []
all_references = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model.generate(input_ids, max_length=64, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
        all_predictions.extend(outputs)
        all_references.extend(labels)

In [55]:
# Decode predictions and references
decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in all_predictions]
decoded_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in all_references]

In [53]:
!pip install rouge_score
!pip install bert_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=88367f23f17b6ac6e810897c820cf3cb46f123296a1f5e284b6c6d1a55a073f1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00


In [56]:
# Compute metrics
from datasets import load_metric
# rouge_types=["rouge1", "rouge2", "rougeL"]
rouge = load_metric("rouge")
# rouge = rouge(rouge_types)
bleu = load_metric("bleu")
from bert_score import score


/tmp/ipykernel_34/1193695119.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


The repository for bleu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bleu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


In [57]:
rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
bleu_result = bleu.compute(predictions=[pred.split() for pred in decoded_preds], references=[[ref.split()] for ref in decoded_labels])
P, R, F1 = score(decoded_preds, decoded_labels, lang='multilingual')
bertscore_result = {"precision": P.mean().item(), "recall": R.mean().item(), "f1": F1.mean().item()}

results = {"bleu": bleu_result, "rouge": rouge_result, "bertscore": bertscore_result}

# Print results
from prettytable import PrettyTable

def print_results(results):
    table = PrettyTable()
    table.field_names = ["Metric", "Value"]

    bleu_scores = results['bleu']
    table.add_row(["BLEU Score", f"{bleu_scores['bleu']:.4f}"])
    table.add_row(["Brevity Penalty", f"{bleu_scores['brevity_penalty']:.4f}"])
    table.add_row(["Length Ratio", f"{bleu_scores['length_ratio']:.4f}"])
    table.add_row(["Translation Length", bleu_scores['translation_length']])
    table.add_row(["Reference Length", bleu_scores['reference_length']])
    for i, precision in enumerate(bleu_scores['precisions'], 1):
        table.add_row([f"Precisions {i}-gram", f"{precision:.4f}"])

    rouge_scores = results['rouge']
    for key in rouge_scores:
        score = rouge_scores[key].mid
        table.add_row([f"ROUGE-{key} (Precision)", f"{score.precision:.4f}"])
        table.add_row([f"ROUGE-{key} (Recall)", f"{score.recall:.4f}"])
        table.add_row([f"ROUGE-{key} (F-Measure)", f"{score.fmeasure:.4f}"])

    bert_scores = results['bertscore']
    table.add_row(["BERTScore Precision", f"{bert_scores['precision']:.4f}"])
    table.add_row(["BERTScore Recall", f"{bert_scores['recall']:.4f}"])
    table.add_row(["BERTScore F1", f"{bert_scores['f1']:.4f}"])

    print(table)

print_results(results)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

+-----------------------------+--------+
|            Metric           | Value  |
+-----------------------------+--------+
|          BLEU Score         | 0.0219 |
|       Brevity Penalty       | 0.8811 |
|         Length Ratio        | 0.8876 |
|      Translation Length     | 11199  |
|       Reference Length      | 12617  |
|      Precisions 1-gram      | 0.0797 |
|      Precisions 2-gram      | 0.0293 |
|      Precisions 3-gram      | 0.0159 |
|      Precisions 4-gram      | 0.0103 |
|   ROUGE-rouge1 (Precision)  | 0.1309 |
|    ROUGE-rouge1 (Recall)    | 0.1167 |
|   ROUGE-rouge1 (F-Measure)  | 0.1110 |
|   ROUGE-rouge2 (Precision)  | 0.0495 |
|    ROUGE-rouge2 (Recall)    | 0.0433 |
|   ROUGE-rouge2 (F-Measure)  | 0.0415 |
|   ROUGE-rougeL (Precision)  | 0.1260 |
|    ROUGE-rougeL (Recall)    | 0.1124 |
|   ROUGE-rougeL (F-Measure)  | 0.1067 |
| ROUGE-rougeLsum (Precision) | 0.1257 |
|   ROUGE-rougeLsum (Recall)  | 0.1125 |
| ROUGE-rougeLsum (F-Measure) | 0.1066 |
|     BERTScore 

In [58]:
test_set

,review_text,review_summary
256368,I like the 2009 reboot of Star Trek. Into Dark...,An okay movie... unfaithful to the characters ...
349866,Thomas Jane was excellent in The Punisher. In ...,The Punisher
473523,How can a movie sell so many tickets when it h...,really disappointing
35606,"I'm an Indian, the first time i saw this movie...",nothing compares
289001,[This comment contains what some people might ...,"Spooky, but sometimes ridiculous"
...,...,...
464829,I did not really plan to watch this movie beca...,Madcap Zany Fun Movie
513967,This film is great it has a bit for everyone i...,the best film you will see
229191,went to watch this movie and was very disappoi...,phoned in
146998,The story focuses on a man who is consumed wit...,Story had one track mind - no substance to plot


In [62]:
input_review = test_set.loc[146998]
input_review['review_text']

"The story focuses on a man who is consumed with self gratification sexually. He is also in inner turmoil to some extent. His sister moves in with him unexpectedly and there are subtle references to her wanting him sexually. Beyond this shallow contrived plot line...there is nothing really going on. The movie just didn't go anywhere. The acting was okay, but there is a scene of his sister singing an entire new york new york song...really. Another scene shows him running in the streets for like 5 minutes. It felt like the director needed to add lots of fluff to this movie to make it 2 hours or whatever time it ran. It was boring and not worth watching in my opinion."

In [63]:
input_review['review_summary']

'Story had one track mind - no substance to plot'

In [65]:

# Example article for prediction
example_article = input_review['review_text']
input_encoding = tokenizer(example_article, return_tensors="pt", max_length=128, padding="max_length", truncation=True)

print("input review:")
print(example_article)

print()

# Generate title
model.eval()
with torch.no_grad():
    input_encoding = {k: v.to(device) for k, v in input_encoding.items()}
    outputs = model.generate(input_encoding['input_ids'], max_length=64, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    predicted_title = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Predicted review summary:")
print(predicted_title)

input review:
The story focuses on a man who is consumed with self gratification sexually. He is also in inner turmoil to some extent. His sister moves in with him unexpectedly and there are subtle references to her wanting him sexually. Beyond this shallow contrived plot line...there is nothing really going on. The movie just didn't go anywhere. The acting was okay, but there is a scene of his sister singing an entire new york new york song...really. Another scene shows him running in the streets for like 5 minutes. It felt like the director needed to add lots of fluff to this movie to make it 2 hours or whatever time it ran. It was boring and not worth watching in my opinion.

Predicted review summary:
Waste of Time
